In [1]:
import pandas as pd
import numpy as np

In [2]:
dfFull = pd.read_csv('ContentMatrix.csv', index_col=0)
dfFull.shape

(60648, 73)

In [3]:
dfFull.head()

,userID,gameName,playHours,purchase,appid,rating,english,developer,publisher,average_playtime,...,Steam Leaderboards,Steam Trading Cards,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,VR Support,Valve Anti-Cheat enabled,linux,mac,windows
0,48028873,100% orange juice,1.8,1.0,282800.0,4.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,88525821,100% orange juice,0.9,1.0,282800.0,2.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,93644606,100% orange juice,3.1,1.0,282800.0,4.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,115996656,100% orange juice,2.5,1.0,282800.0,5.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,125017535,100% orange juice,0.7,1.0,282800.0,3.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [4]:
dfFull.drop_duplicates(subset=['userID', 'gameName'], inplace=True)
names = pd.DataFrame(dfFull[['gameName', 'appid']])
names.drop_duplicates(inplace=True)
names.loc[names.gameName=='left 4 dead 2']

,gameName,appid
30303,left 4 dead 2,550.0


In [5]:
dfFull.loc[dfFull.appid==550].head()

,userID,gameName,playHours,purchase,appid,rating,english,developer,publisher,average_playtime,...,Steam Leaderboards,Steam Trading Cards,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,VR Support,Valve Anti-Cheat enabled,linux,mac,windows
30303,298950,left 4 dead 2,16.300000,1.0,550.0,3.0,1.0,Valve,Valve,1615.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
30304,561758,left 4 dead 2,26.000000,1.0,550.0,3.0,1.0,Valve,Valve,1615.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
30305,975449,left 4 dead 2,69.540094,1.0,550.0,5.0,1.0,Valve,Valve,1615.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
30306,1950243,left 4 dead 2,49.000000,1.0,550.0,5.0,1.0,Valve,Valve,1615.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
30307,2259650,left 4 dead 2,69.540094,1.0,550.0,5.0,1.0,Valve,Valve,1615.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0


In [6]:
# Import all apps list from steam api
AllApps = pd.read_csv('APIdata/AllGames.csv', index_col=0)
AllApps.dropna(inplace=True)
AllApps.drop_duplicates(inplace=True)
AllApps.head()

,appid,name
4,216938,Pieterw test app76 ( 216938 )
5,660010,test2
6,660130,test3
18,508100,Robotpencil Presents: Setting up Lights in a P...
19,508101,Robotpencil Presents: Setting up Lights in a P...


In [7]:
AllApps.shape

(146533, 2)

In [8]:
# Join onto existing appd data, main purpose of this is to get appids for all games in the data set
# this will give me a list of appids for which I can requst app metadata from the steamspy api.
AllApps.name = AllApps.name.str.replace(':', '')
AllApps.name = AllApps.name.str.replace('Â®', '')
AllApps.name = AllApps.name.str.replace('â„¢', '') 
AllApps.name = AllApps.name.str.replace('®', '')
AllApps.name = AllApps.name.str.replace('™', '')
AllApps.name = AllApps.name.str.replace('- ', '')
AllApps.name = AllApps.name.str.replace('-', '')

AllApps.name = AllApps.name.apply(lambda x: x.lower())

In [9]:
AllApps.head()

,appid,name
4,216938,pieterw test app76 ( 216938 )
5,660010,test2
6,660130,test3
18,508100,robotpencil presents setting up lights in a pa...
19,508101,robotpencil presents setting up lights in a pa...


In [10]:
dfFull.shape

(60648, 73)

In [11]:
dfFull = dfFull.merge(AllApps, how='left', left_on='gameName', right_on='name')
dfFull.head()

,userID,gameName,playHours,purchase,appid_x,rating,english,developer,publisher,average_playtime,...,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,VR Support,Valve Anti-Cheat enabled,linux,mac,windows,appid_y,name
0,48028873,100% orange juice,1.8,1.0,282800.0,4.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,282800.0,100% orange juice
1,88525821,100% orange juice,0.9,1.0,282800.0,2.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,282800.0,100% orange juice
2,93644606,100% orange juice,3.1,1.0,282800.0,4.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,282800.0,100% orange juice
3,115996656,100% orange juice,2.5,1.0,282800.0,5.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,282800.0,100% orange juice
4,125017535,100% orange juice,0.7,1.0,282800.0,3.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,282800.0,100% orange juice


In [12]:
dfFull.loc[dfFull.appid_x!=dfFull.appid_y].shape

(14652, 75)

In [13]:
noMatch = dfFull.loc[dfFull.appid_x!=dfFull.appid_y, ['appid_x', 'appid_y', 'gameName']].drop_duplicates()

In [14]:
noMatch.dropna(subset='appid_x', inplace=True)

In [15]:
noMatch.loc[noMatch.appid_x.isna()].head(60)

,appid_x,appid_y,gameName


In [16]:
noMatch.loc[noMatch.appid_y.isna(), 'appid_y'] = noMatch.appid_x

In [17]:
noMatch.head(60)

,appid_x,appid_y,gameName
378,224540.0,224540.0,ace of spades
651,349700.0,582700.0,aftermath
652,349700.0,1705720.0,aftermath
653,349700.0,1916830.0,aftermath
858,105450.0,105450.0,age of empires iii complete collection
1426,10680.0,10699.0,aliens vs. predator
2558,33900.0,33909.0,arma 2
2559,33900.0,33910.0,arma 2
2560,33900.0,33919.0,arma 2
2929,33910.0,33900.0,arma 2


In [18]:
dfFull.loc[dfFull.appid_x.isna(), 'appid_x'] = dfFull.appid_y

In [19]:
dfFull.loc[dfFull.index.isin(noMatch.index), ['gameName', 'appid_x', 'appid_y']].head(60)

,gameName,appid_x,appid_y
378,ace of spades,224540.0,NaN
651,aftermath,349700.0,582700.0
652,aftermath,349700.0,1705720.0
653,aftermath,349700.0,1916830.0
858,age of empires iii complete collection,105450.0,NaN
1426,aliens vs. predator,10680.0,10699.0
2558,arma 2,33900.0,33909.0
2559,arma 2,33900.0,33910.0
2560,arma 2,33900.0,33919.0
2929,arma 2,33910.0,33900.0


### Bring in app feature data collected from the steamspy api

In [20]:
b1 = pd.read_csv('APIdata/AppMetaDataB1.csv', index_col=0)
b2 = pd.read_csv('APIdata/AppMetaDataB2.csv', index_col=0)
b3 = pd.read_csv('APIdata/AppMetaDataB3.csv', index_col=0)

MetaData = pd.concat([b1, b2, b3])
MetaData.fillna(value=0, inplace=True)
MetaData.shape

(732, 381)

In [21]:
MetaData.head()

,appid,developer,publisher,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,3D,...,NSFW,Sailing,Skiing,Snow,Snowboarding,Social Deduction,Video Production,Vikings,Warhammer 40K,Werewolves
0,282800,Orange_Juice,Fruitbat Factory,0.0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,223850,UL,UL,0.0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,251570,The Fun Pimps,The Fun Pimps Entertainment LLC,0.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,250420,Archive Entertainment,Archive Entertainment,0.0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,278360,Gone North Games,Coffee Stain Publishing,0.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
dfFull.drop_duplicates(subset=['userID', 'gameName', 'playHours', 'appid_x'], inplace=True)

There were some duplicate appids for individual games in the data pulled from steams api, resulting in duplicates and issues when joining with steamspy data. A quick pull of data from steamspy's api for these duplicate appids revealed which ones returned valid data and which did not. The invalid appid duplicates were dropped from the data.

In [23]:
fun = dfFull.loc[dfFull.duplicated(subset=['userID', 'gameName', 'playHours'], keep=False)]
fun.gameName.unique()

array(['aftermath', 'bioshock', 'borderlands',
       'call of duty modern warfare 3', 'darksiders',
       'napoleon total war',
       'red orchestra 2 heroes of stalingrad single player',
       "sid meier's civilization iv beyond the sword", 'super meat boy',
       'the last remnant', 'the walking dead',
       'the witcher enhanced edition',
       "tom clancy's splinter cell conviction", 'total war shogun 2',
       'trine', 'universe sandbox'], dtype=object)

In [24]:
fun.loc[fun.gameName=="universe sandbox"].appid_x.value_counts()

80776.0     41
230290.0    41
Name: appid_x, dtype: int64

In [25]:
dfFull.loc[dfFull.gameName=='portal 2', 'appid_x'].value_counts()

620.0    496
Name: appid_x, dtype: int64

In [26]:
# Dropping duplicated invalid appids
dfFull = dfFull.loc[~dfFull.appid_x.isin([582700, 1705720, 7700, 8950, 42699, 115300, 5160, 17312, 2115, 50620, 10540, 
                                           34009, 8662, 669970, 34039, 212541, 8804, 34460, 34450, 40809, 1989070, 23319, 
                                           581510, 9810, 33220, 201270, 35709, 80776])]

In [27]:
dfFull.loc[dfFull.duplicated(subset=['userID', 'gameName'])]

,userID,gameName,playHours,purchase,appid_x,rating,english,developer,publisher,average_playtime,...,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,VR Support,Valve Anti-Cheat enabled,linux,mac,windows,appid_y,name


In [28]:
dfFull.appid_x.isna().sum()

1113

In [29]:
b = dfFull.loc[~dfFull.appid_x.isin(MetaData.appid), ['gameName', 'appid_x']]
b.drop_duplicates(inplace=True)
b.loc[b.appid_x.isna()].head(32)

,gameName,appid_x
332,ace of spades,NaN
766,age of empires ii hd edition,NaN
10217,call of duty modern warfare 2 multiplayer,NaN
11875,company of heroes (new steam version),NaN
18301,divinity original sin,NaN
18732,don't starve together beta,NaN
20431,endless space,NaN
23991,firefall,NaN
24325,fractured space,NaN
24432,freestyle2 street basketball,NaN


In [30]:
# Any remaing games with missing appids are dropped
dfFull.dropna(subset=['appid_x'], inplace=True)

In [31]:
dfFull.loc[(dfFull.appid_x.isna())&(dfFull.appid_y.isna()), ['appid_x', 'appid_y']].sum()

appid_x    0.0
appid_y    0.0
dtype: float64

In [32]:
dfFull.shape

(59458, 75)

In [33]:
dfFull.isna().sum()

userID           0
gameName         0
playHours        0
purchase         0
appid_x          0
             ...  
linux        12302
mac          12302
windows      12302
appid_y       1499
name          1499
Length: 75, dtype: int64

In [34]:
dfFull.drop_duplicates(subset=['userID', 'gameName'], inplace=True)
dfFull.shape

(59458, 75)

In [35]:
dfFull.drop(columns=['appid_y', 'name'], inplace=True)
dfFull.rename(columns={'appid_x':'appid'}, inplace=True)
dfFull.head()

,userID,gameName,playHours,purchase,appid,rating,english,developer,publisher,average_playtime,...,Steam Leaderboards,Steam Trading Cards,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,VR Support,Valve Anti-Cheat enabled,linux,mac,windows
0,48028873,100% orange juice,1.8,1.0,282800.0,4.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,88525821,100% orange juice,0.9,1.0,282800.0,2.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,93644606,100% orange juice,3.1,1.0,282800.0,4.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,115996656,100% orange juice,2.5,1.0,282800.0,5.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,125017535,100% orange juice,0.7,1.0,282800.0,3.0,1.0,Orange_Juice,Fruitbat Factory,1678.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


### Add meta data gathered from steamspy api
Now that all games have been linked to thier appropirate appid we can pull in game feature data gathered from the steamspy api. This data includes tags, genres, developers, and publishers.

In [36]:
dfFull.appid.isna().sum()

0

In [37]:
dfFull.appid.isna().sum()

0

In [38]:
# Export for now, once I have steamspy details, I can come back to this notebook and add them to the data.
# Purpose of this is to limit the calls made to the steamspy api to only games included in the data set used for modeling.
# dfFull.to_csv('ContentMatrix2.csv')

## Import and concatenate the steamspy data

In [39]:
b1 = pd.read_csv('APIdata/AppMetaDataB1.csv', index_col=0)
b2 = pd.read_csv('APIdata/AppMetaDataB2.csv', index_col=0)
b3 = pd.read_csv('APIdata/AppMetaDataB3.csv', index_col=0)
b4 = pd.read_csv('APIdata/AppMetaDataB4.csv', index_col=0)

In [40]:
MetaData = pd.concat([b1, b2, b3, b4])
MetaData.shape

(788, 384)

In [41]:
# Now remove old metadata columns from full df and merge with new meta data
dfFull = dfFull[['userID', 'gameName', 'playHours', 'rating', 'appid']]
dfFull.shape

(59458, 5)

In [42]:
dfFull = dfFull.merge(MetaData, how='left', on='appid')
dfFull.shape

(59458, 388)

In [43]:
dfFull.to_csv('ContentMatrix2.csv')